# CO₂ Concentration at Mauna Loa (Weekly) – Keeling Curve
### Computational document – Time Series Analysis
**Author:** Hanae Tafza  
**Data source:** Scripps CO₂ Program (Mauna Loa Observatory)  
**Download date:** 3 December 2025

In 1958, Charles David Keeling started measuring the atmospheric CO₂ concentration at Mauna Loa Observatory (Hawaii, USA). These measurements are known as the **Keeling Curve**.

In this computational document, we will:

1. Plot the raw weekly CO₂ series and show the **superposition** of a fast **seasonal oscillation** and a slower **long-term trend**.
2. **Separate** these two components using seasonal decomposition (trend + seasonal + residual).
3. **Characterize the periodic oscillation** (amplitude, pattern).
4. Fit a **simple linear model** for the slow trend, estimate its parameters, and **extrapolate** it until 2025.


## 1. Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams["figure.figsize"] = (14, 6)

## 2. Loading the Weekly Scripps CO₂ Data

We use the **weekly in situ CO₂ data** from the Scripps CO₂ Program at Mauna Loa.  
Official page: <https://scrippsco2.ucsd.edu/data/atmospheric_co2/weekly_in_situ_co2/>

For this document, we download the CSV directly from the Scripps website.  
In a real reproducible workflow, we would store a **local copy** in the Git repository with the download date (here: 3 December 2025).

In [ ]:
url = "https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2_mlo_weekly.csv"

# The file contains commented header lines starting with '#'
df_raw = pd.read_csv(url, comment="#")
df_raw.head()

Let us inspect the structure of the dataset to identify the relevant columns (year, date, CO₂ value, etc.).

In [ ]:
df_raw.info()

## 3. Cleaning and Preprocessing

The Scripps weekly file typically contains columns such as:

- `year` or `Yr`
- possibly `month` / `day` or `week` number
- `decimal` (decimal year)
- `value` = weekly average CO₂ (ppm), with missing values encoded as **-99.99**
- `interpolated` or `trend` may also be provided

We will:
- choose a CO₂ column (prefer `value` if present, otherwise `interpolated`),
- remove or interpolate missing values,
- build a proper `datetime` index for weekly observations.


In [ ]:
# Try to detect the main CO2 column
candidates = ["value", "co2", "interpolated", "interp"]
co2_col = None
for c in candidates:
    if c in df_raw.columns:
        co2_col = c
        break

if co2_col is None:
    raise ValueError("Could not find a CO2 column in the dataset. Please check df_raw.columns.")

co2_col

In [ ]:
# Copy to a working DataFrame
df = df_raw.copy()
df.rename(columns={co2_col: "co2"}, inplace=True)

# Handle missing values: Scripps usually encodes them as -99.99
df["co2"] = df["co2"].replace(-99.99, np.nan)

# If there is an 'interpolated' column, we can use it to fill missing values
interp_col = None
for c in ["interpolated", "interp"]:
    if c in df_raw.columns:
        interp_col = c
        break

if interp_col is not None:
    df["co2"] = df["co2"].fillna(df_raw[interp_col])
else:
    # Fallback: interpolate over time if needed
    df["co2"] = df["co2"].interpolate()

df[["co2"]].describe()

### Constructing a Datetime Index

The file may contain either:
- `year` + `week` columns, or
- `year` + `month` + `day`, or
- a `decimal` year column.

We will try, in order:
1. If `year` and `week` exist: use them to build weekly dates.
2. Else if `year`, `month`, `day` exist: build dates directly.
3. Else if `decimal` exists: approximate dates using decimal year.


In [ ]:
cols = df.columns

if "year" in cols and "week" in cols:
    # Year + ISO week -> approximate by adding (week-1)*7 days
    df["year"] = df["year"].astype(int)
    df["week"] = df["week"].astype(int)
    df["date"] = pd.to_datetime(df["year"].astype(str), format="%Y") + \
                  pd.to_timedelta((df["week"] - 1) * 7, unit="D")
elif all(c in cols for c in ["year", "month", "day"]):
    df["date"] = pd.to_datetime(df[["year", "month", "day"]].astype(int))
elif "decimal" in cols:
    # Approximate from decimal year
    df["decimal"] = df["decimal"].astype(float)
    years = df["decimal"].astype(int)
    frac = df["decimal"] - years
    df["date"] = pd.to_datetime(years.astype(str), format="%Y") + \
                  pd.to_timedelta(frac * 365.25, unit="D")
else:
    raise ValueError("No suitable date columns found. Please inspect df_raw.columns.")

df = df.sort_values("date").reset_index(drop=True)
df[["date", "co2"]].head()

We now have a cleaned weekly time series of CO₂ concentration with a proper timestamp.

For the decomposition, we will set `date` as the index and use an **additive model** with a period of 52 weeks (approx. one year).

In [ ]:
df_ts = df.set_index("date")
# Ensure regular frequency (weekly); if not, we can resample to weekly and interpolate
df_ts = df_ts.asfreq("W-MON")  # assume Monday weekly frequency
df_ts["co2"] = df_ts["co2"].interpolate()
df_ts.head()

## 4. Raw CO₂ Time Series – Superposition of Seasonal and Trend

We first plot the full weekly CO₂ series to visually confirm the **superposition of a periodic oscillation** and a **slowly increasing trend**.

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(df_ts.index, df_ts["co2"], label="Weekly CO₂ (ppm)", color="darkred")
plt.title("Weekly Atmospheric CO₂ at Mauna Loa – Raw Series")
plt.xlabel("Year")
plt.ylabel("CO₂ (ppm)")
plt.legend()
plt.tight_layout()
plt.show()

### Interpretation
- The CO₂ series clearly exhibits a **long-term upward trend**: concentration has increased strongly since 1958.
- Superimposed on this trend, we observe a **regular oscillation** with a period of about one year.
- This seasonal oscillation is linked to the **carbon cycle** of the biosphere, especially vegetation in the Northern Hemisphere.

This plot fulfills the first objective: it shows the **superposition of a fast periodic component and a slow systematic evolution**.

## 5. Seasonal Decomposition (Trend + Seasonal + Residual)

We now use `seasonal_decompose` from `statsmodels` to separate:

- **Trend**: slow multi-year evolution of CO₂
- **Seasonal**: yearly oscillation
- **Residual**: remaining noise/irregularities

We assume a **weekly series** with a period of **52** weeks per year, and use an **additive model**:  
`co2(t) = trend(t) + seasonal(t) + residual(t)`.

In [ ]:
result = seasonal_decompose(df_ts["co2"], model="additive", period=52, extrapolate_trend="freq")

trend = result.trend
seasonal = result.seasonal
resid = result.resid

fig = result.plot()
fig.set_size_inches(14, 10)
plt.tight_layout()
plt.show()

### Interpretation
- **Trend**: the trend panel isolates the smooth, long-term increase in CO₂. It removes seasonal fluctuations and reveals a nearly monotonic upward curve.
- **Seasonal**: the seasonal panel shows a **regular annual cycle** with peaks and troughs repeating every year and a fairly stable shape over time.
- **Residual**: the residual captures irregular variations (noise, measurement errors, short-term phenomena).

This decomposition explicitly separates the two phenomena requested in the assignment:  
1. The **periodic oscillation** (seasonal component)  
2. The **slow systematic evolution** (trend component).


## 6. Characterizing the Periodic Oscillation

We now quantify the seasonal component:
- amplitude (difference between maximum and minimum)
- typical timing of peaks and troughs over the year.


In [ ]:
# Seasonal component over one year (average across all years)
seasonal_df = seasonal.to_frame(name="seasonal").dropna().copy()
seasonal_df["week_of_year"] = seasonal_df.index.isocalendar().week

# Average seasonal effect by week of year
weekly_seasonal = seasonal_df.groupby("week_of_year")["seasonal"].mean()

amp = weekly_seasonal.max() - weekly_seasonal.min()
amp, weekly_seasonal.idxmax(), weekly_seasonal.idxmin()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(weekly_seasonal.index, weekly_seasonal.values, marker="o")
plt.title("Average Seasonal CO₂ Cycle (Weekly)")
plt.xlabel("Week of Year")
plt.ylabel("Seasonal Component (ppm)")
plt.tight_layout()
plt.show()

### Interpretation
- The **amplitude** of the seasonal component (max–min) is on the order of a few ppm (the precise value is printed in the previous cell).
- The **maximum** seasonal CO₂ typically occurs in late **spring** (around the week with the highest seasonal value).
- The **minimum** seasonal CO₂ occurs in late **summer** or early **autumn**.

This pattern corresponds to the biological carbon cycle:
- During the growing season in the Northern Hemisphere, vegetation absorbs CO₂ (concentration decreases).
- During autumn and winter, leaves fall and organic matter decomposes, releasing CO₂ (concentration increases).


## 7. Modeling the Long-Term Trend and Extrapolating to 2025

We now focus on the **trend component** obtained from the decomposition.  
We approximate it by a **simple linear model**:

$$ \text{trend}(t) = a \cdot t + b $$

where $t$ is the decimal year.  
We will:

1. Build a decimal time axis from the dates.
2. Fit a linear regression on the (time, trend) pairs.
3. Use this model to **extrapolate until the end of 2025**.


In [ ]:
# Prepare data for regression: drop NaNs in the trend
trend_df = trend.dropna().to_frame(name="trend")
trend_df["year"] = trend_df.index.year
trend_df["day_of_year"] = trend_df.index.dayofyear
trend_df["t"] = trend_df["year"] + (trend_df["day_of_year"] - 0.5) / 365.25  # decimal year

# Linear regression: trend ~ a * t + b
x = trend_df["t"].values
y = trend_df["trend"].values
a, b = np.polyfit(x, y, 1)
a, b

The slope `a` (in ppm/year) represents the **average annual increase** in CO₂ concentration according to our simple linear model.  
The intercept `b` is the modeled CO₂ value at time `t = 0` (not directly meaningful physically, but needed for the equation).

We now use this model to compute the fitted trend over the historical period and extrapolate it beyond the last observation up to 2025.

In [ ]:
# Build a combined time axis: historical + extrapolation to 2025
last_date = df_ts.index.max()

# Time axis for historical data
t_hist = trend_df["t"].values
trend_fit_hist = a * t_hist + b

# Time axis for future weekly points up to 2025-12-31
future_dates = pd.date_range(start=last_date, end="2025-12-31", freq="W-MON")
future_year = future_dates.year
future_doy = future_dates.dayofyear
t_future = future_year + (future_doy - 0.5) / 365.25
trend_future = a * t_future + b

# Plot
plt.figure(figsize=(14,6))
plt.plot(trend_df.index, trend_df["trend"], label="Decomposed trend (observed)", color="black")
plt.plot(trend_df.index, trend_fit_hist, label="Linear fit on trend", color="blue")
plt.plot(future_dates, trend_future, "r--", label="Extrapolation to 2025")
plt.title("CO₂ Trend Component with Linear Model and Extrapolation")
plt.xlabel("Year")
plt.ylabel("CO₂ (ppm)")
plt.legend()
plt.tight_layout()
plt.show()

### Interpretation
- The **linear model** fits the decomposed trend reasonably well over the historical period.
- The slope `a` gives a typical growth rate in **ppm/year**. It has increased over decades in reality, but a linear approximation is acceptable as a simple model.
- The extrapolation until **2025** shows continued growth of atmospheric CO₂.

To validate the model, we can compare the extrapolated values with the actual observations for the years close to 2025 when they become available.  
If the real measurements deviate strongly from the extrapolated trend, this indicates that a **more complex (non-linear) model** may be necessary (e.g., quadratic or exponential trend).

## 8. Summary and Conclusions

In this computational document, we analyzed the weekly CO₂ series from Mauna Loa (Keeling Curve) using time series methods:

1. **Superposition of components**: The raw time series shows a clear combination of a **seasonal oscillation** and a **slow, systematic increase**.
2. **Separation of phenomena**: Using seasonal decomposition (additive model, period = 52 weeks), we separated the series into **trend**, **seasonal**, and **residual** components.
3. **Seasonal characterization**: The seasonal component has an amplitude of a few ppm and shows a regular annual cycle: CO₂ peaks in late spring and is lowest in late summer, reflecting the terrestrial carbon cycle in the Northern Hemisphere.
4. **Trend modeling**: We fitted a **simple linear model** to the decomposed trend. The slope corresponds to the average annual increase in CO₂ concentration.
5. **Extrapolation to 2025**: We extrapolated the linear model to 2025, providing a simple forecast of future CO₂ levels. This can be used as a baseline to compare with future observations.

This exercise illustrates how time series analysis can separate and characterize different components of a signal and how simple models can be used for trend estimation and extrapolation, while keeping in mind their limitations in a complex system such as the Earth's climate.